<!--NOTEBOOK_HEADER-->
*This notebook contains material from [Controlling Natural Watersheds](https://jckantor.github.io/Controlling-Natural-Watersheds);
content is available [on Github](https://github.com/jckantor/Controlling-Natural-Watersheds.git).*

<!--NAVIGATION-->
< [Data Sources](http://nbviewer.jupyter.org/github/jckantor/Controlling-Natural-Watersheds/blob/master/notebooks/A.00-Data_Sources.ipynb) | [Contents](toc.ipynb) | [Water Survey of Canada Usage](http://nbviewer.jupyter.org/github/jckantor/Controlling-Natural-Watersheds/blob/master/notebooks/A.02-Water_Survey_of_Canada_Usage.ipynb) ><p><a href="https://colab.research.google.com/github/jckantor/Controlling-Natural-Watersheds/blob/master/notebooks/A.01-Water_Survey_of_Canada_HYDAT_Database.ipynb"><img align="left" src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open in Colab" title="Open in Google Colaboratory"></a><p><a href="https://raw.githubusercontent.com/jckantor/Controlling-Natural-Watersheds/master/notebooks/A.01-Water_Survey_of_Canada_HYDAT_Database.ipynb"><img align="left" src="https://img.shields.io/badge/Github-Download-blue.svg" alt="Download" title="Download Notebook"></a>

# Water Survey of Canada HYDAT Database

Historical data for lake levels and stream flows in the Rainy River watershed are available from public sources including the [HYDAT database](https://www.ec.gc.ca/rhc-wsc/default.asp?lang=En&n=9018B5EC-1) from the [Water Survey of Canada (WSC)](http://wateroffice.ec.gc.ca/). The [HYDAT database](https://www.ec.gc.ca/rhc-wsc/default.asp?lang=En&n=9018B5EC-1) database consists of Microsoft Access .mdb file providing data on daily water flows and levels throughout Canada.

For the purposes of these notebooks, individual tables (STATIONS, DLY_FLOWS, and DLY_LEVELS) were extracted from the database as .csv files with [MDB Explorer](http://www.mdbexplorer.com/). This notebook extracts information relevant to the Rainy River basin from the .csv file and stores the results in the repository's data directory.  Due to size constraints, only the data relevant to the Rainy River Watershed is included in the repository. The HYDAT database and extracted .csv files are omitted from the repository.

New versions of the HYDAT database normally become available on a quarterly basis.  

## Initialization

Load needed libaries and data tables from the HYDAT database.

In [1]:
# Standard Python modules
import pandas as pd
from IPython.core.display import display
from IPython.display import IFrame

## Read HYDAT data

In [2]:
# Directory where data files have been stored
HYDAT_dir = "../data/HYDAT/csv/"

# Read data files as a pandas dataframes
HYDAT_STATIONS = pd.read_csv(HYDAT_dir + 'STATIONS.csv', index_col = 0);
HYDAT_DLY_LEVELS = pd.read_csv(HYDAT_dir + 'DLY_LEVELS.csv')
HYDAT_DLY_FLOWS = pd.read_csv(HYDAT_dir + 'DLY_FLOWS.csv')

print("    Stations in the HYDAT database = ", len(HYDAT_STATIONS.index))

    Stations in the HYDAT database =  7842


## Locating Monitoring Stations in the Rainy River Watershed

The following cell creates a pandas dataframe of monitoring stations from the STATIONS.csv table extracted from the HYDAT database. The extaction searches for all stations with a specified region bounded by latitude and longitudes.

For reference, this is a map of the [Rainy River drainage](http://www.ijc.org/files/tinymce/uploaded/rl_basinmap.pdf) basin available from the International Joint Commission. 

![](../figures/IJC/rl_basinmap_640.png)

In [3]:
# bounding region
lngW = -94.0
lngE = -90.5
latS = 47.5
latN = 49.0

# find monitoring stations within bounding region
stationList = HYDAT_STATIONS[
    (HYDAT_STATIONS['LATITUDE'] >= latS) & \
    (HYDAT_STATIONS['LATITUDE'] <= latN) & \
    (HYDAT_STATIONS['LONGITUDE'] <= lngE) & \
    (HYDAT_STATIONS['LONGITUDE'] >= lngW)].index
STATIONS = HYDAT_STATIONS.loc[stationList]

DLY_LEVELS = HYDAT_DLY_LEVELS.loc[HYDAT_DLY_LEVELS['STATION_NUMBER'].isin(stationList)]
DLY_FLOWS = HYDAT_DLY_FLOWS.loc[HYDAT_DLY_FLOWS['STATION_NUMBER'].isin(stationList)]

# add columns to STATIONS for level and flow stations
STATIONS['Level'] = ''
STATIONS['Flow'] = ''

# mark level and flow stations
STATIONS.loc[set(DLY_LEVELS['STATION_NUMBER']).intersection(STATIONS.index),'Level'] = True
STATIONS.loc[set(DLY_FLOWS['STATION_NUMBER']).intersection(STATIONS.index),'Flow'] = True

print("Stations within region of interest = ", len(STATIONS.index))
print("DLY_FLOWS records = ", len(DLY_FLOWS.index))
print("DLY_LEVELS records = ", len(DLY_LEVELS.index))
display(STATIONS.loc[:,['Level','Flow','STATION_NAME','LATITUDE','LONGITUDE',]])

STATIONS.to_pickle('../data/HYDAT_STATIONS')

Stations within region of interest =  34
DLY_FLOWS records =  8289
DLY_LEVELS records =  6381


,Level,Flow,STATION_NAME,LATITUDE,LONGITUDE
STATION_NUMBER,,,,,
05PA001,,True,KETTLE RIVER ABOVE KETTLE FALLS,48.49444,-92.64444
05PA003,True,,NAMAKAN LAKE ABOVE KETTLE FALLS DAM,48.50000,-92.63886
05PA005,True,,NORTHERN LIGHT LAKE AT OUTLET,48.24028,-90.76889
05PA006,True,True,NAMAKAN RIVER AT OUTLET OF LAC LA CROIX,48.38256,-92.17631
05PA007,True,,CROOKED LAKE NEAR CURTAIN FALLS,48.23750,-91.90611
05PA010,True,,FRENCH LAKE NEAR ATIKOKAN,48.67222,-91.13500
05PA011,True,,LAC LA CROIX AT CAMPBELL'S CAMP,48.35508,-92.21744
05PA012,True,True,BASSWOOD RIVER NEAR WINTON,48.08256,-91.65117
05PA013,True,,NAMAKAN LAKE AT SQUIRREL ISLAND,48.49686,-92.65856


## Pickle Level Data to LEVELS

Extract level data from the HYDAT database and pickle to LEVELS data set. The pickled dataframe is indexed by date with columns tagged by station name.

In [5]:
def getLEVELS(s):
    '''Return a time series for levels at a given station.'''
    ts = {}   
    data = DLY_LEVELS[DLY_LEVELS['STATION_NUMBER'] == s]
    for k in data.index:
        mo = str(data.loc[k,'MONTH'])
        yr = str(data.loc[k,'YEAR'])
        for n in range(1,data.loc[k,'NO_DAYS']+1):
            ts[pd.to_datetime(mo + '/' + str(n) + '/' + yr)] = data.loc[k, 'LEVEL'+str(n)]  
    ts = pd.Series(ts)
    #drop initial and terminal null entries
    j = 0
    while j < len(ts.index) and pd.isnull(ts.iloc[j]):
        j += 1
    k = len(ts.index) - 1
    while k>=j and pd.isnull(ts.iloc[k]):
        k += -1
    return ts[j:k+1]

LEVELS = pd.DataFrame({s:getLEVELS(s) for s in STATIONS[STATIONS['Level']==True].index})
LEVELS.to_pickle('../data/HYDAT_LEVELS')
LEVELS.tail()

,05PA003,05PA005,05PA006,05PA007,05PA010,05PA011,05PA012,05PA013,05PB002,05PB007,05PB012,05PB014,05PB018,05PB023,05PB024,05PB025,05PC018,05PC022,05PC024,05PC025
2018-12-27,NaN,NaN,360.978,NaN,NaN,NaN,NaN,NaN,NaN,337.164,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,337.119,NaN
2018-12-28,NaN,NaN,360.982,NaN,NaN,NaN,NaN,NaN,NaN,337.164,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,337.120,NaN
2018-12-29,NaN,NaN,360.977,NaN,NaN,NaN,NaN,NaN,NaN,337.166,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,337.102,NaN
2018-12-30,NaN,NaN,360.971,NaN,NaN,NaN,NaN,NaN,NaN,337.163,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,337.094,NaN
2018-12-31,NaN,NaN,360.966,NaN,NaN,NaN,NaN,NaN,NaN,337.155,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,337.086,NaN


## Pickle Flow Data to FLOWS

The DLY_FLOW data is encoded in an irregular tabular format where rows are indexed by station code, year, and month, and columns are indexed by date.  Given a station code, the following function decodes DLY_FLOW to produce a pandas times series of flow rates.  The function is used to create a pandas dataframe for all flow stations in STATIONS, then pickles the results to a data file for use by other notebooks in this repository.

In [6]:
def getFLOWS(s):
    ts = {}
    data = DLY_FLOWS[DLY_FLOWS['STATION_NUMBER'] == s]
    for k in data.index:
        mo = str(data.loc[k,'MONTH'])
        yr = str(data.loc[k,'YEAR'])
        for n in range(1,data.loc[k,'NO_DAYS']+1):
            ts[pd.to_datetime(mo + '/' + str(n) +' /' + yr)] = data.loc[k, 'FLOW'+str(n)]  
    ts = pd.Series(ts)
    ts.name = s +': ' + STATIONS.loc[s,'STATION_NAME'] + \
        ' from ' + '{0}'.format(ts.index[0].year) + \
        ' to ' + '{0}'.format(ts.index[-1].year)
    #drop initial and terminal null entries
    j = 0
    while j < len(ts.index) and pd.isnull(ts.iloc[j]):
        j += 1
    k = len(ts.index) - 1
    while k>=j and pd.isnull(ts.iloc[k]):
        k += -1
    return ts[j:k+1]

FLOWS = pd.DataFrame({s: getFLOWS(s) for s in STATIONS[STATIONS['Flow']==True].index})
FLOWS.to_pickle('../data/HYDAT_FLOWS')
FLOWS.tail()

,05PA001,05PA006,05PA012,05PB001,05PB003,05PB004,05PB009,05PB014,05PB015,05PB018,05PB019,05PB020,05PB021,05PB022,05PC009,05PC010,05PC016,05PC018,05PC019,05PC022
2018-12-27,NaN,102.0,NaN,NaN,NaN,NaN,48.4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,242.0,NaN
2018-12-28,NaN,103.0,NaN,NaN,NaN,NaN,47.8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,243.0,NaN
2018-12-29,NaN,102.0,NaN,NaN,NaN,NaN,47.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,279.0,NaN
2018-12-30,NaN,101.0,NaN,NaN,NaN,NaN,39.6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,280.0,NaN
2018-12-31,NaN,101.0,NaN,NaN,NaN,NaN,42.3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,280.0,NaN


<!--NAVIGATION-->
< [Data Sources](http://nbviewer.jupyter.org/github/jckantor/Controlling-Natural-Watersheds/blob/master/notebooks/A.00-Data_Sources.ipynb) | [Contents](toc.ipynb) | [Water Survey of Canada Usage](http://nbviewer.jupyter.org/github/jckantor/Controlling-Natural-Watersheds/blob/master/notebooks/A.02-Water_Survey_of_Canada_Usage.ipynb) ><p><a href="https://colab.research.google.com/github/jckantor/Controlling-Natural-Watersheds/blob/master/notebooks/A.01-Water_Survey_of_Canada_HYDAT_Database.ipynb"><img align="left" src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open in Colab" title="Open in Google Colaboratory"></a><p><a href="https://raw.githubusercontent.com/jckantor/Controlling-Natural-Watersheds/master/notebooks/A.01-Water_Survey_of_Canada_HYDAT_Database.ipynb"><img align="left" src="https://img.shields.io/badge/Github-Download-blue.svg" alt="Download" title="Download Notebook"></a>